# Példa Minta Szálloda és Repülőjegy Foglaló Ügynök

Ez a megoldás segít repülőjegyek és szállodák foglalásában. A forgatókönyv egy utazás London Heathrow LHR 2024. február 20-tól New York JFK-ig, visszatérés 2025. február 27-én, economy osztályon csak British Airways járattal. Szeretnék egy szállást egy Hilton hotelben New Yorkban, kérlek adj meg árakat a repülőre és a szállodára.


# Inicializálja az Azure AI Agent Service-t és szerezze be a konfigurációs információkat a **.env** fájlból

### **.env** 

Hozzon létre egy .env fájlt

A **.env** tartalmazza az Azure AI Agent Service kapcsolati karakterláncát, az AOAI által használt modellt, valamint a megfelelő Google API Search szolgáltatás API-ját, ENDPOINT-ját stb.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Az Ön Azure AI Agent Service Modell Telepítésének Neve"

[**MEGJEGYZÉS**] Olyan modellre lesz szüksége, amely 100 000-es Rate Limit-tel (token per perc) és 600-as Rate Limit-tel (kérés per perc) rendelkezik.

  A modellt a Microsoft Foundry - Model and Endpoint alatt szerezheti be.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Az Ön Azure AI Agent Service Projekt Kapcsolati Karakterlánca"

  A projekt kapcsolati karakterláncot az AI Foundry Portál képernyőjén, a projekt áttekintésében találhatja meg.

- **SERPAPI_SEARCH_API_KEY** = "Az Ön SERPAPI Search API KULCSA"
- **SERPAPI_SEARCH_ENDPOINT** = "Az Ön SERPAPI Search Endpoint-ja"

Az Azure AI Agent Service Modell Telepítési Nevének és Projekt Kapcsolati Karakterláncának megszerzéséhez létre kell hoznia az Azure AI Agent Service-t. Ajánlott [ezt a sablont](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) használni a közvetlen létrehozáshoz (***Megjegyzés:*** Az Azure AI Agent Service jelenleg korlátozott régióban érhető el. Ajánlott, hogy hivatkozzon [erre a linkre](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) a régió beállításához)

Az Agent-nek hozzáférést kell kapnia a SERPAPI-hoz. Ajánlott [ezen a linken](https://serpapi.com/searches) keresztül regisztrálni. A regisztráció után egy egyedi API KULCSOT és ENDPOINT-ot kaphat.


# Beállítás

A jegyzetfüzet futtatásához győződj meg róla, hogy telepítetted a szükséges könyvtárakat a `pip install -r requirements.txt` parancs futtatásával.


In [ ]:
from semantic_kernel import __version__

__version__

A Semantic Kernel verziójának legalább 1.27.2-nek kell lennie.


Töltse be a .env fájl beállításait és erőforrásait, kérjük, győződjön meg róla, hogy hozzáadta a kulcsait és beállításait, valamint hogy létrehozott egy helyi .env fájlt.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Jelentkezzen be az Azure-ba

Most be kell jelentkeznie az Azure-ba. Nyisson meg egy terminált, és futtassa a következő parancsot:

```bash
az login
```

Ez a parancs kérni fogja, hogy adja meg Azure-hitelesítő adatait, lehetővé téve az Azure AI Agent szolgáltatás helyes működését.


# Magyarázat:
Ez egy változó, amely egy SERP (Keresőmotor találati oldal) API szolgáltatáshoz való hozzáféréshez szükséges API kulcsot tárolja. Az API kulcs egy egyedi azonosító, amelyet a fiókodhoz kapcsolódó kérések hitelesítésére használnak.

Cél: Ennek a sorozatnak a célja, hogy az API kulcsot egy változóban tárolja, hogy azt a SERP API szolgáltatás hitelesítésére használd. Az API kulcs szükséges a szolgáltatás eléréséhez és a keresések végrehajtásához.
Hogyan szerezhető SERP API kulcs: Ahhoz, hogy SERP API kulcsot kapj, kövesd a https://serpapi.com oldalon található általános lépéseket (a pontos lépések eltérhetnek a használt konkrét SERP API szolgáltatástól függően):

Válassz SERP API szolgáltatást: Több SERP API szolgáltatás is elérhető, mint például a SerpAPI, a Google Custom Search JSON API és mások. Válaszd ki azt, amelyik leginkább megfelel az igényeidnek.

Regisztrálj egy fiókot: Látogass el a kiválasztott SERP API szolgáltatás weboldalára, és hozz létre egy fiókot. Előfordulhat, hogy néhány alapvető információt kell megadnod és az e-mail címedet igazolnod kell.

Hozz létre egy API kulcsot: Regisztráció után jelentkezz be a fiókodba, és menj az API szekcióhoz vagy irányítópulthoz. Keresd meg az új API kulcs létrehozására vagy generálására szolgáló opciót.
Másold be az API kulcsot a .env fájlodba.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Ez egy változó, amely a SERP API végpont alap URL-jét tárolja. A BASE_URL változónév egy konvenció, amely azt jelzi, hogy ez az URL az API-kérések kiindulópontja.
'https://serpapi.com/search':

Ez a tényleges URL karakterlánc, amely a BASE_URL változóhoz van rendelve. Ez jelenti a végpontot a keresési lekérdezések végrehajtásához a SERP API használatával.

# Purpose:
Ennek a sornak a célja egy olyan konstans definiálása, amely tartalmazza a SERP API alap URL-jét. Ezt az URL-t fogják használni az API-kérések összeállításának kiindulópontjaként a keresési műveletek végrehajtásához.

# Usage:
Az alap URL változóba helyezésével könnyen újra felhasználhatja azt a kód több pontján, amikor csak kérést kell küldeni a SERP API-nak. Ez a kód karbantarthatóságát növeli, és csökkenti a hibák kockázatát, amely a több helyen keményen kódolt URL használatakor fordulhat elő. A jelenlegi példa https://serpapi.com/search?engine=bing, amely a Bing kereső API-t használja. Különböző API választható a https://Serpapi.com oldalon.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Magyarázat:

Itt található a plugin-kódod.

Osztály definíció: `class BookingPlugin`: Egy BookingPlugin nevű osztály definiálása, amely módszereket tartalmaz hotelek és járatok foglalására.

Szállás foglalási metódus:

- `@kernel_function(description="booking hotel")`: Egy díszítő, amely leírja a funkciót, mint a hotelek foglalásához tartozó kernel-funkciót.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Egy metodus definiálása hotelek foglalására, annotált paraméterekkel és visszatérési típussal.

A metódus összeállít egy paraméter szótárat a hotel foglalási kéréshez és egy GET kérést küld a SERP API-nak. Ellenőrzi a válasz állapotát, és visszaadja a hotel tulajdonságokat, ha sikeres, vagy None-t, ha a kérés sikertelen volt.

Repülőjegy foglalási metódus:

- `@kernel_function(description="booking flight")`: Egy díszítő, amely leírja a funkciót, mint a repülőjegyek foglalásához tartozó kernel-funkciót.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Egy metódus definiálása repülőjegyek foglalására annotált paraméterekkel és visszatérési típussal.

A metódus összeállítja a paraméter szótárakat a kimenő és a visszaúti járatok kéréséhez, és GET kéréseket küld a SERP API-nak. Ellenőrzi a válasz állapotát, és hozzáfűzi a repülőjárat információkat az eredmény sztringhez, ha sikeres, vagy hibát jelez, ha a kérés sikertelen volt. A metódus visszaadja az eredmény sztringet, amely tartalmazza a repülőjárat információkat.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Magyarázat:
Importálási kijelentések: Szükséges modulok importálása az Azure hitelesítő adatok, AI ügynök, csevegési üzenetek tartalma, szerzői szerep és kernel függvény dekorátor számára.

Aszinkron Kontextuskezelő: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Ez egy aszinkron kontextuskezelőt állít be az Azure hitelesítő adatok kezelésére és az AI ügynök kliens létrehozására.

Ügynök neve és utasítások:
- `AGENT_NAME = "BookingAgent"`: Meghatározza az ügynök nevét.
- `AGENT_INSTRUCTIONS = """..."""`: Részletes utasításokat ad az ügynök számára a foglalási kérelmek kezeléséhez.

Ügynök definíció létrehozása: `agent_definition = await client.agents.create_agent(...)`: Ügynök definíció létrehozása a megadott modell, név és utasítások alapján.

AzureAI Ügynök létrehozása: `agent = AzureAIAgent(...)`: AzureAI ügynök létrehozása a kliens, az ügynök definíció és a definiált bővítmény segítségével.

Szál létrehozása: `thread: AzureAIAgentThread | None = None`: Ügynök szál létrehozása. Nem szükséges először szálat létrehozni – ha a `None` érték van megadva, az első hívás során új szál jön létre és válaszként visszaadásra kerül.

Felhasználói bemenetek: `user_inputs = ["..."]`: A feldolgozandó felhasználói bemenetek listája az ügynök számára.

A finally blokkban töröld a szálat és az ügynököt az erőforrások felszabadításához.


# Hitelesítés

A `DefaultAzureCredential` osztály az Azure SDK for Python része. Alapértelmezett módot biztosít az Azure szolgáltatásokhoz való hitelesítéshez. Több módszerrel próbál hitelesíteni meghatározott sorrendben, például környezeti változók, kezelt identitás és Azure CLI hitelesítő adatok használatával.

Aszinkron műveletek: Az aio modul jelzi, hogy a DefaultAzureCredential osztály támogatja az aszinkron műveleteket. Ez azt jelenti, hogy használhatja az asyncio-val blokkmentes hitelesítési kérelmek végrehajtásához.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Jogi nyilatkozat**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordító szolgáltatásával készült. Bár a pontosságra törekszünk, kérjük, vegye figyelembe, hogy az automatikus fordítások tartalmazhatnak hibákat vagy pontatlanságokat. Az eredeti dokumentum az anyanyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt a professzionális emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
